In [1]:
import pandas as pd

train_df = pd.read_csv('Train.csv')
test_df = pd.read_csv('Test.csv')

In [2]:
train_df

,Category,IncidentGrade,EntityType,EvidenceRole,IpAddress,Url,AccountName,DayOfWeek,Month,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7
0,10,2,15,1,27,160396,453297,1,6,0,1,1,1,1,1,1
1,7,1,32,0,360606,160396,24887,4,6,1,1,1,0,1,1,1
2,10,1,31,1,360606,68652,453297,3,6,1,1,1,1,1,1,1
3,1,0,31,1,360606,13,453297,0,6,1,1,1,1,1,1,1
4,6,2,32,0,360606,160396,479,5,6,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9465492,10,2,6,1,360606,160396,453297,5,6,1,1,1,1,1,1,1
9465493,1,0,31,1,360606,164,453297,3,6,1,1,1,1,1,1,1
9465494,7,0,21,0,360606,160396,453297,1,6,1,1,1,0,0,1,1
9465495,9,0,15,1,465,160396,453297,0,5,1,1,1,1,1,1,1


In [3]:
test_df

,Category,IncidentGrade,EntityType,EvidenceRole,IpAddress,Url,AccountName,DayOfWeek,Month,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7
0,11,0,28,0,360606,160396,863,1,6,1,1,1,0,1,1,1
1,1,0,15,0,360606,160396,453297,0,6,1,2,0,1,0,1,1
2,11,0,23,1,360606,160396,453297,5,6,1,1,0,1,1,1,0
3,10,1,7,1,360606,160396,453297,2,6,1,1,1,1,1,1,1
4,5,0,28,0,360606,160396,136104,3,6,1,1,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4147987,10,0,28,0,360606,160396,14567,1,6,1,1,1,0,1,1,1
4147988,10,0,28,0,360606,160396,107151,1,6,1,1,1,0,1,1,1
4147989,1,1,12,1,1084,160396,453297,5,6,1,1,1,1,1,1,1
4147990,10,0,18,0,360606,160396,25428,1,6,1,1,1,0,1,1,1


In [4]:
import pandas as pd
from scipy.stats import zscore



z_scores = train_df.apply(zscore)

in_range = (z_scores >= -3) & (z_scores <= 3)

rows_in_range = in_range.all(axis=1)

train_df = train_df[rows_in_range]

num_rows = train_df.shape[0]

print(f"Number of rows with z-scores between -3 and 3 for all columns: {num_rows}")

Number of rows with z-scores between -3 and 3 for all columns: 7545335


In [5]:
import pandas as pd
from scipy.stats import zscore



z_scores = test_df.apply(zscore)

in_range = (z_scores >= -3) & (z_scores <= 3)

rows_in_range = in_range.all(axis=1)

test_df = test_df[rows_in_range]

num_rows = test_df.shape[0]

print(f"Number of rows with z-scores between -3 and 3 for all columns: {num_rows}")

Number of rows with z-scores between -3 and 3 for all columns: 3291584


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline  # Import Pipeline from sklearn.pipeline
from sklearn.metrics import classification_report

# Assuming train_df is already defined
X = train_df.drop('IncidentGrade', axis=1)
y = train_df['IncidentGrade']

# Define the ColumnTransformer
numerical_transformer = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median'))            
        ]), X.columns)
    ])

# Define the pipeline with LGBMClassifier
pipeline = ImbPipeline(steps=[
    ('preprocessor', numerical_transformer),
    ('under_sampler', RandomUnderSampler(random_state=42)),
    ('classifier', LGBMClassifier(random_state=42))
])

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Perform cross-validation with cv=3
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=4, scoring='accuracy')
print("Cross-Validation Scores:", cv_scores)
print("Mean Cross-Validation Score:", cv_scores.mean())

# Fit the pipeline
pipeline.fit(X_train, y_train)

# Predict and evaluate on training data
y_pred_train = pipeline.predict(X_test)
report_train = classification_report(y_test, y_pred_train, target_names=['Class 0', 'Class 1', 'Class 2'])
print("Training Data Classification Report:\n", report_train)

# Predict and evaluate on test data
X_test_df = test_df.drop('IncidentGrade', axis=1)
y_test_df = test_df['IncidentGrade']
y_pred_test_df = pipeline.predict(X_test_df)
report_test_df = classification_report(y_test_df, y_pred_test_df, target_names=['Class 0', 'Class 1', 'Class 2'])
print("Test Data Classification Report:\n", report_test_df)



[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.079622 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 829
[LightGBM] [Info] Number of data points in the train set: 2468658, number of used features: 12
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.069285 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 828
[LightGBM] [Info] Number of data points in the train set: 2468655, number of used features: 12
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Star